In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
## read in the dataset
df_weather, df_load = pd.read_csv('raw/weather_info_spain.csv'), pd.read_csv('raw/electricity_dataset_spain.csv')
df_weather['dt_iso'] = pd.to_datetime(df_weather['dt_iso']).apply(lambda x: pd.Timestamp(x).tz_localize(None))
df_load['time'] = pd.to_datetime(df_load['time']).apply(lambda x: pd.Timestamp(x).tz_localize(None))
df_load.rename(columns={'time':'dt_iso','price day ahead': 'price_forecast', 'price actual': 'price_actual', 'total load forecast': 'load_forecast', 'total load actual': 'load_actual'},inplace=True)

df_weather = df_weather.groupby(['dt_iso']).agg({'temp':['min','median','mean','max'], 'humidity':['min','median','mean','max']})
df_weather.columns = [f'{x[0]}_{x[1]}' for x in df_weather.columns]

In [3]:
## collect the hf block
df_hf = pd.merge(df_weather, df_load[['dt_iso','price_forecast','price_actual']],on='dt_iso',how='inner')
## shift price_baseline by one-day since that's when it becomes available; shift weather by one hour
df_hf['price_forecast'] = df_hf['price_forecast'].shift(-24)
for col in ['temp','humidity']:
    for ops in ['min','median','mean','max']:
        df_hf[f'{col}_{ops}'] = df_hf[f'{col}_{ops}'].shift(-1)
## take logarithm for price
df_hf['price_forecast'] = np.log(df_hf['price_forecast']+1e-6)
df_hf['price_actual'] = np.log(df_hf['price_actual']+1e-6)
## add +1hr to the timestamp, so that it corresponds to the end
df_hf['dt_iso'] = df_hf['dt_iso'].apply(lambda x: x + pd.Timedelta(hours=+1))
df_hf = df_hf.set_index('dt_iso').dropna()

**check for missing timestamps for the high-frequency data**

In [4]:
timestamp_hf_list = [df_hf.index.min() + datetime.timedelta(hours=x) for x in range(int(24*((df_hf.index.max()-df_hf.index.min()).days+1)))]

In [5]:
hf_missing_stamps = set(timestamp_hf_list) - set(df_hf.index)
print(f'missing stamps for hf: {hf_missing_stamps}')

missing stamps for hf: {Timestamp('2016-03-27 03:00:00'), Timestamp('2017-03-26 03:00:00'), Timestamp('2018-03-25 03:00:00'), Timestamp('2015-03-29 03:00:00')}


In [6]:
df_hf_missing = pd.DataFrame(index = list(hf_missing_stamps), columns=df_hf.columns)
df_hf = pd.concat([df_hf, df_hf_missing]).sort_index().ffill()
df_hf.index.name = 'timestamp'

In [7]:
print(f'duplicated stamps for hf: {df_hf.index[df_hf.index.duplicated()]}')

duplicated stamps for hf: DatetimeIndex(['2015-10-25 03:00:00', '2016-10-30 03:00:00',
               '2017-10-29 03:00:00', '2018-10-28 03:00:00'],
              dtype='datetime64[ns]', name='timestamp', freq=None)


In [8]:
df_hf = df_hf.loc[~df_hf.index.duplicated()].copy()

In [9]:
## collect the lf block
df_lf = df_load[['dt_iso','load_forecast','load_actual']].copy()
## shift load_forecast by 24 hr
df_lf['load_forecast'] = df_lf['load_forecast'].shift(-24)
## create lf timestamp
df_lf['dt_6H'] = df_lf['dt_iso'].apply(lambda x: (x + pd.Timedelta(hours=+1)).ceil("6H"))
df_lf = df_lf.groupby(['dt_6H']).agg({'load_forecast':'sum','load_actual':'sum'})
## logarithm of load
df_lf['load_forecast'] = np.log(df_lf['load_forecast']+1).replace(to_replace=0, method='ffill')
df_lf['load_actual'] = np.log(df_lf['load_actual']+1).replace(to_replace=0, method='ffill')
df_lf.index.name = 'timestamp'

In [10]:
timestamp_lf_list = set([x.ceil("6H") for x in timestamp_hf_list])

In [11]:
lf_missing_stamps = set(timestamp_lf_list) - set(df_lf.index)
print(f'missing stamps for lf: {lf_missing_stamps}')

missing stamps for lf: set()


In [12]:
print(f'duplicated stamps for lf: {df_lf.index[df_lf.index.duplicated()]}')

duplicated stamps for lf: DatetimeIndex([], dtype='datetime64[ns]', name='timestamp', freq=None)


In [13]:
df_lf = df_lf.loc[df_lf.index <= df_hf.index.max()].copy()

In [14]:
with pd.ExcelWriter(f'electricity.xlsx') as writer:
    df_hf.to_excel(writer,sheet_name='x',index=True)
    df_lf.drop(columns=['load_forecast']).to_excel(writer,sheet_name='y',index=True)